In [1]:
from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import pickle

with open('fixing_model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('fixing_data', 'rb') as file:
    data = pickle.load(file)

full_ft_names = ['Number Of Lines Added', 'Number Of Modified Directories', 'Number Of Modified Subsystems', 'Entropy', 'Number Of Review Revisions', 'Review Timespan', 'Number of Self Approvals', 'Number of Past Developers', 'Time Since Last Modification', 'Number of Reviewrs', 'Relative Reviewer Experience', 'Author Awareness', 'Reviewer Awareness']    

explainer = PyExplainer(X_train=data['x_train'],
                       y_train=data['y_train'],
                       dep=data['dep'],
                       indep=data['indep'],
                       blackbox_model=model,
                       full_ft_names=full_ft_names)

In [2]:
import pandas as pd


commit_id = pd.read_csv('fixing_csv.csv')
commit_ids = commit_id['commit_id'].to_list()
print('Choose one row from index 0 to index', len(commit_id)-1, 'to be explained')
print('specify your choice into index variable')

# risky index - 20 34 154 19 7 21 30 34 36 60 61 62 63 67 70 93 117
############################
####### specify here #######
############################
index = 154

print(commit_ids[index])

Choose one row from index 0 to index 197 to be explained
specify your choice into index variable
a9a59ccbacafd6eb94f57861cfc28f5a24f474db


In [3]:
d = pd.read_csv('fixing_csv.csv', index_col='commit_id')

X_explain = d.loc[[commit_ids[index]]]

y_test = d.loc[:, data['dep']]
y_explain = y_test.loc[[commit_ids[index]]]

for col in X_explain.columns:
    if col not in data['indep']:
        X_explain = X_explain.drop([col], axis=1)
        
rules = explainer.explain(X_explain=X_explain, y_explain=y_explain)
explainer.visualise(rules)

Output(layout=Layout(border='3px solid black'))

FloatSlider(value=3.0, continuous_update=False, description='#1 The value of Number of Reviewrs is less than 3…

FloatSlider(value=3.0, continuous_update=False, description='#2 The value of Number Of Modified Directories is…

FloatSlider(value=70.0, continuous_update=False, description='#3 The value of Number of Past Developers is mor…

In [4]:
pd.set_option('display.max_colwidth', None)
print(rules['top_k_negative_rules']['rule'])

659            la > 8.425000190734863 & la > 22.280000686645508 & app <= 3.0649999380111694
336     rsawr <= 0.38499999046325684 & la > 124.53499984741211 & rrexp <= 1849.135009765625
1762                                    app <= 3.9250000715255737 & la <= 62.14999961853027
Name: rule, dtype: object


In [4]:
explainer.bullet_data

[{'title': '#1 The value of Number Of Modified Directories is less than 3',
  'subtitle': 'Actual = 3',
  'ticks': [0.0, 4.5],
  'step': [0.1],
  'startPoints': [0, 150.0],
  'widths': [150.0, 300.0],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [3],
  'varRef': 'nd'},
 {'title': '#2 The value of Number Of Review Revisions is less than 12.0',
  'subtitle': 'Actual = 12.0',
  'ticks': [1.8000000000000007, 17.1],
  'step': [1],
  'startPoints': [0, 150.0],
  'widths': [150.0, 300.0],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [12.0],
  'varRef': 'nrev'},
 {'title': '#3 The value of Time Since Last Modification is less than 0.67',
  'subtitle': 'Actual = 0.67',
  'ticks': [0.67, 0.67],
  'step': [0.1],
  'startPoints': [0, 0],
  'widths': [0, 0],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [0.67],
  'varRef': 'age'}]

In [4]:
from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import pickle
from pyexplainer import pyexplainer_pyexplainer


with open('fixing_model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('fixing_data', 'rb') as file:
    data = pickle.load(file)

full_ft_names = ['Number Of Lines Added', 'Number Of Modified Directories', 'Number Of Modified Subsystems', 'Entropy', 'Number Of Review Revisions', 'Review Timespan', 'Number of Self Approvals', 'Number of Past Developers', 'Time Since Last Modification', 'Number of Reviewrs', 'Relative Reviewer Experience', 'Author Awareness', 'Reviewer Awareness']    

explainer = PyExplainer(X_train=data['x_train'],
                       y_train=data['y_train'],
                       dep=data['dep'],
                       indep=data['indep'],
                       blackbox_model=model,
                       full_ft_names=full_ft_names)
exp_rule = explainer.explain(X_explain=X_explain, y_explain=y_explain)
#with open("rules.pkl", 'wb+') as file:
#    pickle.dump(obj=rules, file=file)
#with open("fixing_rules.pkl", 'rb') as file:
#    rules = pickle.load(file)
    
exp_rule2 = explainer.explain(X_explain=X_explain, y_explain=y_explain)

In [ ]:
rules.keys()

In [12]:
pd.set_option('display.max_colwidth', None)
rules['top_k_positive_rules'].head(100)

,index,rule,type,coef,support,importance,is_satisfy_instance
0,573,ent > 0.009999999776482582 & rsawr <= 0.5450000166893005 & asawr > 0.014999999664723873,rule,0.243041,0.451697,0.120952,True
1,53,app > 3.009999990463257 & ent <= 1.024999976158142 & la > 16.78499984741211,rule,0.169330,0.313316,0.078542,True
2,681,la > 33.63999938964844 & asawr > 0.03499999921768904,rule,0.156090,0.315927,0.072564,True
3,707,nrev > 0.9399999976158142 & nrev <= 4.640000104904175 & rsawr <= 0.574999988079071,rule,0.256845,0.083551,0.071072,True
4,493,rrexp <= 2637.8450927734375 & age > 0.4099999964237213 & la > 3.435000002384186,rule,0.143778,0.668407,0.067689,True
...,...,...,...,...,...,...,...
95,231,app > 2.0049999952316284 & app <= 2.975000023841858 & app <= 3.915000081062317,rule,0.017349,0.248042,0.007493,True
96,646,app > 1.8149999976158142 & app <= 3.9850000143051147 & app > 1.2099999785423279,rule,0.016414,0.704961,0.007486,True
97,247,la <= 18.179999351501465,rule,0.020478,0.156658,0.007443,True
98,569,app <= 2.9850000143051147 & app <= 2.0549999475479126,rule,0.016550,0.214099,0.006789,True


In [8]:
rules['top_k_negative_rules'].head(10)

,rule,type,coef,support,importance,Class
1447,la <= 102.36999893188477 & nrev > 4.0550000667...,rule,-0.216527,0.569191,0.107222,Clean
916,app <= 4.009999990463257 & age > 0.09500000253...,rule,-0.239525,0.736292,0.105545,Clean
255,app <= 4.045000076293945 & app > 3.98000001907...,rule,-0.312617,0.109661,0.097682,Clean


In [15]:
X_explain.head(1)

,la,nd,ns,ent,nrev,rtime,self,ndev,age,app,rrexp,asawr,rsawr
commit_id,,,,,,,,,,,,,
a9a59ccbacafd6eb94f57861cfc28f5a24f474db,155,3,1,0.736602,12.0,32.516586,0,70.0,0.669703,3.0,2374.0,0.143199,0.390069


In [16]:
exp_rule['top_k_positive_rules'].head(10)

,index,rule,type,coef,support,importance,is_satisfy_instance
0,749,rsawr <= 0.5450000166893005 & rtime > -10.554999828338623 & app <= 4.9649999141693115,rule,0.181072,0.783290,0.074602,True
1,220,app <= 3.990000009536743 & app > 3.115000009536743 & la > 63.56999969482422,rule,0.222603,0.125326,0.073701,True
2,147,nd > 1.6350000500679016 & app <= 2.9200000762939453,rule,0.145486,0.409922,0.071553,True
3,318,ent > 0.07999999821186066 & nd <= 1.3849999904632568 & ent <= 0.9650000035762787,rule,0.209754,0.070496,0.053693,True
4,622,la > 20.244999885559082 & age > 0.08500000089406967,rule,0.113795,0.712794,0.051488,True
5,110,la > 11.485000133514404 & la > 106.56499862670898 & rtime > 12.394999980926514,rule,0.110490,0.237598,0.047026,True
6,142,app > 1.4150000214576721 & nd <= 1.6649999618530273 & ent <= 0.9449999928474426,rule,0.152068,0.093995,0.044377,True
7,356,age > 0.13499999791383743 & app > 1.1500000357627869 & nrev > 1.199999988079071,rule,0.098119,0.738903,0.043097,True
8,281,nrev > 1.0250000357627869 & asawr <= 0.024999999441206455 & age <= 14.384999752044678,rule,0.085863,0.467363,0.042840,True
9,35,nrev > 1.4050000309944153 & la > 47.685001373291016 & ent <= 0.9449999928474426,rule,0.086260,0.563969,0.042775,True


In [17]:
exp_rule['top_k_negative_rules'].head(10)

,rule,type,coef,support,importance,Class
1675,rtime <= 21.539999961853027 & rtime > 2.9950000047683716,rule,-0.195445,0.475196,0.097602,Clean
329,la > 36.05999946594238 & app <= 3.0049999952316284,rule,-0.193577,0.428198,0.095785,Clean
1076,la <= 56.13999938964844 & nd <= 3.884999990463257 & ent <= 0.9750000238418579,rule,-0.200250,0.328982,0.094086,Clean


In [5]:
top_r = explainer.parse_top_rules(exp_rule['top_k_positive_rules'], exp_rule['top_k_negative_rules'])
top_r['top_tofollow_rules']

[{'variable': 'age', 'lessthan': False, 'value': '0.4099999964237213'},
 {'variable': 'la', 'lessthan': False, 'value': '131.55500030517578'}]

In [22]:
data['x_train'].max()

la       80629.000000
nd         304.000000
ns           6.000000
ent          1.000000
nrev        82.000000
rtime      372.608553
self         1.000000
ndev       354.000000
age        649.346238
app         20.000000
rrexp     3749.000000
asawr        1.000000
rsawr        1.000000
dtype: float64

In [6]:
t_rule = top_r['top_tofollow_rules'][0]
float(t_rule['value'])

0.4099999964237213

In [7]:
explainer.X_explain = X_explain
explainer.bullet_data = explainer.generate_bullet_data(parsed_rule_object=top_r)
explainer.temp_func(explainer.generate_html())

AttributeError: 'PyExplainer' object has no attribute 'temp_func'

In [8]:
explainer.bullet_data

[{'title': '#1 The value of Time Since Last Modification is less than 0.67',
  'subtitle': 'Actual = 0.67',
  'ticks': [0, 1.34],
  'step': [0.1],
  'startPoints': [0, 137.6866],
  'widths': [137.6866, 312.3134],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [0.67],
  'varRef': 'age'},
 {'title': '#2 The value of Number Of Lines Added is less than 155',
  'subtitle': 'Actual = 155',
  'ticks': [0, 310],
  'step': [1],
  'startPoints': [0, 190.9669],
  'widths': [190.9669, 259.0331],
  'colors': ['#d7191c', '#a6d96a'],
  'markers': [155],
  'varRef': 'la'}]

In [ ]:
top_r2 = explainer.parse_top_rules(exp_rule['top_k_positive_rules'], exp_rule['top_k_negative_rules'])
print(top_r)
print(top_r2)

In [18]:
data.keys()

dict_keys(['x_train', 'y_train', 'indep', 'dep'])

In [20]:
data['dep']

'defect'

In [31]:
with open('explained_instance.pkl', 'wb+') as file:
    pickle.dump(obj=a, file=file)

In [27]:
a = {'X_explain': X_explain, 'y_explain': y_explain}


dict_keys(['X_explain', 'y_explain'])

In [33]:
type(y_explain)

pandas.core.series.Series